In [1]:
import holoviews as hv; hv.extension('bokeh', logo=False)
import panel as pn;     pn.extension('katex')

import numpy as np
import pandas as pd
import sympy as sym

<div style="float:center;width:100%;text-align: center;"><strong style="height:60px;color:darkred;font-size:40px;">Numerical Solution of Partial Differential Equations</strong></div>

# 1. Numerical Approximation of Derivatives

## 1.1 First Derivative

The definition of the derivative of a function $f(x)$ is given by<br>
$\qquad \frac{d f}{dx}(x) = \lim_{\Delta x \rightarrow 0} \frac{f(x+\Delta x) - f(x) }{\Delta x},\;\;$ i.e.,<br><br>
$\qquad \frac{d f}{dx}(x) \approx \frac{f(x+\Delta x) - f(x) }{\Delta x}\;\;$ for sufficiently small $\Delta x$.

In [2]:
# Numerical Example
def f(x): return np.cos(x**2)
def df_dx(x): return -np.sin(x**2)*2*x

x  = np.arange(0, 1.05, .05)
dx = [1., .8, .6, .4, .2, .05]

df = pd.DataFrame( {'dx' : dx, 'slope' : [ (f(h)-f(0))/h for h in dx]})

h = hv.Curve( (x,f(x))).opts(title="Estimate the Slope of a Curve") *\
    hv.Curve( ((0,1),(f(0),f(1)))) *\
    hv.Curve( ((0,.8),(f(0),f(.8)))) *\
    hv.Curve( ((0,.6),(f(0),f(.6)))) *\
    hv.Curve( ((0,.4),(f(0),f(.4))))

pn.Row( h,
        pn.Column( pn.pane.Str("Estimated slope at x = 0", styles={'font-size': '12pt'}), pn.pane.DataFrame(df),
                   pn.pane.Str("The smaller dx, the better the approximation", styles={'font-size': '12pt'})
        ))

Row
    [0] HoloViews(Overlay, height=300, sizing_mode='fixed', width=300)
    [1] Column
        [0] Str(str, styles={'font-size': '12pt'})
        [1] DataFrame(DataFrame)
        [2] Str(str, styles={'font-size': '12pt'})

## 1.2 Second Derivative

We can approximate a second derivative by<br>
$\qquad \frac{d^2 f}{dx^2}(x) = lim_{\Delta x\rightarrow 0} \frac{ f'(x+\Delta  x) - f'(x)}{\Delta x}$

Using the previous approximation for $f'(x)$, we obtain<br>
$\qquad \frac{d^2 f}{dx^2} (x) \approx \frac{\left( f(x+\Delta x) - f(x) \right) - \left( f(x) - f(x-\Delta x) \right)}{\Delta x^2} = \frac{ f(x+\Delta x)-2 f(x) +f(x-\Delta x)}{\Delta x^2}$

Other approximations are possible, see
[Wikipedia: Numerical differentiation](https://en.wikipedia.org/wiki/Numerical_differentiation) and [D. Levy: Numerical Differentiation](https://math.umd.edu/~dlevy/classes/amsc466/lecture-notes/differentiation-chap.pdf)

**Remark:** These approximations trivially generalize to partial derivatives, e.g., <br><br>
$\qquad \frac{ \partial f}{\partial x} (x,y) = \lim_{\Delta x \rightarrow 0} \frac{ f(x+\Delta x, y) - f(x,y)}{\Delta x} \approx \frac{ f(x+\Delta x, y) - f(x,y)}{\Delta x}\;\;$ for sufficiently small $\Delta x$.

# 2. Example: Discretize the Poisson Equation

## 2.1 Discretize the Problem

<div style="float:left;width:40%;">

Consider $\;\;\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = f(x,y)$<br>
$\qquad$ on the unit square $0 \le x \le 1, 0 \le y \le 1$.<br><br>

Let us define a grid of values: $\left\{ \begin{aligned} x_i &= i\ \Delta x \\ y_j &= j\ \Delta y \\ u_{i j} &= u( x_i,\ y_j ) \end{aligned} \right.$<br><br>
where $\Delta x$ and $\Delta y$
are chosen step sizes for the appoximations of the partial derivatives.<br> For simplicity, we will choose $\Delta x = \Delta y = h$ for some given step size $h$.

Thus, $\;\;\left\{\begin{aligned}
\frac{\partial^2}{\partial x^2} u_{i j} & \approx \frac{1}{h^2} \left( u_{i+1 j} - 2 u_{i j} + u_{i-1 j}\right) \\
\frac{\partial^2}{\partial y^2} u_{i j} & \approx \frac{1}{h^2} \left( u_{i j+1} - 2 u_{i j} + u_{i j-1}\right) \\
\end{aligned}\right.$
</div>
<div style="float:left;width:40%;padding-left:3cm;">
<img src="./Figs/pde_grid.png" width=300>
</div>

Adding and reordering the coordinates, we get
$\;\;\frac{\partial^2}{\partial x^2} u_{i j} + \frac{\partial^2}{\partial y^2} u_{i j} \approx
\frac{1}{h^2}\left( u_{i j-1}+u_{i-1 j} -4 u_{i j} + u_{i+1 j}+u_{i j+1}
    \right)$

For simplicity, choose $h=0.25$, so that the grid has size $5 \times 5$ as shown in the previous figure.
The interior points of the grid then result in the following equations:<br>

$\qquad
\left. \begin{aligned}
    u_{1 0}+u_{0 1} -4 u_{1 1}+u_{2 1}+u_{1 2}  = h^2 f_{1 1} \\
    u_{2 0}+u_{1 1} -4 u_{2 1}+u_{3 1}+u_{2 2}  = h^2 f_{2 1} \\
    u_{3 0}+u_{2 1} -4 u_{3 1}+u_{4 1}+u_{3 2}  = h^2 f_{3 1} \\
%
    u_{1 1}+u_{0 2} -4 u_{1 2}+u_{2 2}+u_{1 3}  = h^2 f_{1 1} \\
    u_{2 1}+u_{1 2} -4 u_{2 2}+u_{3 2}+u_{2 3}  = h^2 f_{2 1} \\
    u_{3 1}+u_{2 2} -4 u_{3 2}+u_{4 2}+u_{3 3}  = h^2 f_{3 1} \\
%
    u_{1 2}+u_{0 3} -4 u_{1 3}+u_{2 3}+u_{1 4}  = h^2 f_{1 1} \\
    u_{2 2}+u_{1 3} -4 u_{2 3}+u_{3 3}+u_{2 4}  = h^2 f_{2 1} \\
    u_{3 2}+u_{2 3} -4 u_{3 3}+u_{4 3}+u_{3 4}  = h^2 f_{3 1} \\
\end{aligned} \right.
$

We still need to add the boundary conditions, e.g., the following Dirichlet conditions<br><br>
$\qquad \left.\begin{aligned}
    u(x,0)&= \;\;\sin(2\pi x), \quad &  0 \leq x \leq 1, \quad & \text{ lower edge}\\
    u(x,1)&= \;\;\sin(2\pi x),       & 0 \leq x \leq 1,  \quad & \text{ upper edge}\\
    u(0,y)&=2 \sin(2\pi y),          & 0 \leq y \leq 1,  \quad & \text{ left edge}\\
    u(1,y)&=2 \sin(2\pi y),          & 0 \leq y \leq 1,  \quad & \text{ right edge}
    \end{aligned}\quad\right\}
\quad \Rightarrow \quad \left\{
\begin{aligned}\quad
&u_{i 0}   &=\;\; \sin(2\pi x_i)  \\
&u_{i N-1} &=\;\; \sin(2\pi x_i)\\
&u_{0 j}   &= 2 \sin(2\pi y_j) \\
&u_{N-1 j} &= 2 \sin(2\pi y_j)\\
\end{aligned}
\right.$

**We now have the desired number of $N^2$ equations for $N^2$ unknowns.**

Since the Dirichlet conditions specify the values on the boundaries, we can substitute these values into the interior equations directly,<br>
$\qquad$ leaving us with a system of equations that ony involve the interior points and known right hand sides:

$\qquad
\left. \begin{aligned}
& -4 u_{1 1} +  u_{2 1}  +  u_{1 2}                               &=&\; h^2 f_{1 1} - u_{1 0} - u_{0 1}   \\
&    u_{1 1} -4 u_{2 1}  +  u_{3 1} +  u_{2 2}                    &=&\; h^2 f_{2 1} - u_{2 0}             \\
&    u_{2 1} -4 u_{3 1}  +  u_{3 2}                               &=&\; h^2 f_{3 1} - u_{3 0} - u_{4 1}   \\
%
&    u_{1 1} -4 u_{1 2} +   u_{2 2} +  u_{1 3}                    &=&\; h^2 f_{1 1} - u_{0 2}             \\
&    u_{2 1} +  u_{1 2}  -4 u_{2 2} +  u_{3 2} + u_{2 3}          &=&\; h^2 f_{2 1}                       \\
&    u_{3 1} +  u_{2 2}  -4 u_{3 2} +  u_{3 3}                    &=&\; h^2 f_{3 1} - u_{4 2}             \\
%
&    u_{1 2} -4 u_{1 3}  +  u_{2 3}                               &=&\; h^2 f_{1 1} - u_{0 3} -  u_{1 4}  \\
&    u_{2 2} +  u_{1 3}  -4 u_{2 3} +  u_{3 3}                    &=&\; h^2 f_{2 1} - u_{2 4}             \\
&    u_{3 2} +  u_{2 3}  -4 u_{3 3}                               &=&\; h^2 f_{3 1} - u_{4 3}  - u_{3 4}  \\
\end{aligned} \right.
$

## 2.2 Python Implementation

In [3]:
def generate_coefficient_matrix(N):
    ''' Grid of size NxN, order variabels u[1,1], u['''
    N2  = N - 2   # number of interior points in the x and y directions
    Neq = N2*N2
    A   = np.zeros((Neq,Neq))

    for i in range(N2):                  # Diagonal
        for j in range(N2):
            A[i+N2*j,i+N2*j] = -4

    for i in range(1,N2):                # Lower Diagonal
        for j in range(N2):
            A[i+N2*j,i+N2*j-1] = 1

    for i in range(N2):                  # Lower identity
        for j in range(1,N2):
            A[i+N2*j,i+N2*(j-1)] = 1

    for i in range(N2-1):                # Upper Diagonal
        for j in range(N2):
            A[i+N2*j,i+N2*j+1] = 1

    for i in range(N2):                  # Uper Identity
        for j in range(N2-1):
            A[i+N2*j,i+N2*(j+1)] = 1

    return A

In [4]:
N  = 10
A  = generate_coefficient_matrix(N)
hv.Raster(A).opts(cmap="gray", xaxis=None, yaxis=None, width=400, height=400, title="The coefficient matrix is sparse!")

:Raster   [x,y]   (z)

In [5]:
def generate_righthand_side( N, func = lambda i,j: 0):
    ''' func(i,j) yields the boundary condition at grid point (i,j)
    Note we hardcoded the boundary conditions
    '''
    N2     = N-2
    N_vars = N2*N2
    h      = 1. / (N-1)
    h2     = h*h
    a      = 2*np.pi*h

    r      =  np.zeros(N_vars)
    for i in range(N2):                                       # h^2 f_ij term
        for j in range(N2):
            r[i+N2*j] = - h2*func(i,j)

    b_left_right = np.zeros(N_vars)
    for j in range(N2):
        b_left_right[N2*j]         = 2*np.sin(a*(j+1))        # Left edge
        b_left_right[N2-1+N2*j]    = 2*np.sin(a*(j+1))        # Right edge

    b_bottom_top = np.zeros(N_vars)
    for i in range(N2):
        b_bottom_top[i]            = np.sin(a*(i+1))          # Bottom edge
        b_bottom_top[i+N2*(N2-1)]  = np.sin(a*(i+1))          # Top edge

    return r - b_left_right - b_bottom_top

## 2.3 Solve the Resulting Problem

The resulting $A x = b$ style problem is sparse, i.e., most of the entries in $A$ are zero.<br>
Actual solvers take advantage of this fact by **using iterative techniques.**

Here, we will simply use a builtin solver.

In [6]:
N = 64
A = generate_coefficient_matrix(N)
b = generate_righthand_side(N, lambda i,j : 0.01*i*j)

u =  np.linalg.solve(A, b)
u = u.reshape(N-2,-1)
hv.Image(u.T, bounds=(0,0,1,1) ).opts(height=350,width=450, cmap="fire", colorbar=True, title = "Numerical Solution of the Poisson Equation")

:Image   [x,y]   (z)